In [100]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from datetime import datetime
from itertools import accumulate

# Import API key
from config import ATTOM_API

In [101]:
fl_census_data = pd.read_csv('Resources/ACSST5Y2019.S1901_data_with_overlays_2021-05-24T165952.csv', header=1)
fl_codes = []

for i,j in fl_census_data.iterrows():
    fl_codes.append(j['Geographic Area Name'][6:11])

len(fl_codes)

zip_code_split = [123,123,123,123,123,123,123,123]

zip_codes = [list(islice(fl_codes, elem))
            for elem in zip_code_split]

In [102]:
print(zip_codes[0])

['32003', '32008', '32009', '32011', '32024', '32025', '32026', '32033', '32034', '32038', '32040', '32043', '32044', '32046', '32052', '32053', '32054', '32055', '32058', '32059', '32060', '32061', '32062', '32063', '32064', '32065', '32066', '32068', '32071', '32072', '32073', '32079', '32080', '32081', '32082', '32083', '32084', '32086', '32087', '32091', '32092', '32094', '32095', '32096', '32097', '32102', '32110', '32112', '32113', '32114', '32117', '32118', '32119', '32124', '32127', '32128', '32129', '32130', '32131', '32132', '32133', '32134', '32136', '32137', '32139', '32140', '32141', '32145', '32147', '32148', '32157', '32159', '32162', '32164', '32168', '32169', '32174', '32176', '32177', '32179', '32180', '32181', '32187', '32189', '32190', '32193', '32195', '32202', '32204', '32205', '32206', '32207', '32208', '32209', '32210', '32211', '32212', '32216', '32217', '32218', '32219', '32220', '32221', '32222', '32223', '32224', '32225', '32226', '32227', '32228', '32233', 

In [103]:
count = 0
for zip in zip_codes[0]:
    count += 1

print(count)

123


In [104]:
# Read in historical mortgage rate
mortgage_data = pd.read_csv('Resources/morgage_rates.csv')
year_mortgage = []
month_mortgage = []
mortgage_rate = []

# Loop to limit data for 2019 & 2020
for i,j in mortgage_data.iterrows():
    if j['DATE']>='2019%' and j['DATE']<'2021%':
        year_mortgage.append(pd.to_datetime(j['DATE']).year)
        month_mortgage.append(pd.to_datetime(j['DATE']).month)
        mortgage_rate.append(j['MORTGAGE30US'])


# Create mortgage rate DF grouped by year and month to get the mean
mortgage_rates = pd.DataFrame({"year":year_mortgage, "month":month_mortgage, 
                        "mortgage_rate":mortgage_rate}).groupby(['year', 'month'])

# Export csv of calculated monthly rates
mortgage_rates.mean().to_csv('Resources/monthly_morgage_rates.csv')

In [155]:
# Establish API URL
property_url = "https://api.gateway.attomdata.com/propertyapi/v1.0.0/"

sales_trend = "salestrend/snapshot?geoid=ZI"

sales_trend_dates = "&interval=monthly&startyear=2019&endyear=2020&startmonth=january&endmonth=december"

headers = {'Accept': 'application/json', 'apikey': ATTOM_API}

test_zip = ['32608','32601', '32603']

# Establish variable for information desired
year = []
month = []
zip_code_sales = []
total_sales = []
average_price = []
median_price = []
sales_no_data = []

for code in test_zip:
    # Make API call
    sales_response = requests.post( f'{property_url}{sales_trend}{code}{sales_trend_dates}',
                                    headers = headers)
    
    if sales_response.status_code != 200:
        sales_no_data.append(area_response.url)
    else:
        sales = sales_response.json()

    # Loop through API data and append to variables
    for i in sales['salestrends']:
        year.append(pd.to_datetime(i['daterange']['start']).year)
        month.append(pd.to_datetime(i['daterange']['start']).month)
        zip_code_sales.append(i['location']['geoID'][2:7])
        total_sales.append(i['SalesTrend']['homesalecount'])
        average_price.append(i['SalesTrend']['avgsaleprice'])
        median_price.append(i['SalesTrend']['medsaleprice'])
        
    # Set delay based on API call limitations
    time.sleep(7)

# Create new DF with data
sales_data = pd.DataFrame({"zipcode": zip_code_sales, "year":year, "month":month, 
                        "total_sales":total_sales, "avg_sale_price":average_price,
                        "med_sale_price":median_price})

sales_data

,zipcode,year,month,total_sales,avg_sale_price,med_sale_price
0,32608,2019,1,50,353490.0,153550.0
1,32608,2019,3,42,224033.0,163500.0
2,32608,2019,4,95,218566.0,184300.0
3,32608,2019,5,102,221271.0,174950.0
4,32608,2019,6,91,229545.0,171000.0
...,...,...,...,...,...,...
67,32603,2020,8,3,400767.0,375000.0
68,32603,2020,9,2,298506.0,298506.0
69,32603,2020,10,5,365200.0,351000.0
70,32603,2020,11,5,336500.0,350000.0


In [110]:
rates = pd.read_csv('Resources/monthly_morgage_rates.csv')

final_sales = pd.merge(sales_data, rates, on=['year','month'])

final_sales

,zipcode,year,month,total_sales,avg_sale_price,med_sale_price,mortgage_rate
0,32608,2019,1,50,353490.0,153550.0,4.464
1,32601,2019,1,17,148412.0,105000.0,4.464
2,32603,2019,1,5,255607.0,270000.0,4.464
3,32608,2019,3,42,224033.0,163500.0,4.265
4,32601,2019,3,18,194111.0,183750.0,4.265
...,...,...,...,...,...,...,...
67,32601,2020,11,14,265121.0,220000.0,2.765
68,32603,2020,11,5,336500.0,350000.0,2.765
69,32608,2020,12,79,232935.0,221000.0,2.684
70,32601,2020,12,15,208810.0,159900.0,2.684


In [179]:
# Establish API URL
community_url = "https://api.gateway.attomdata.com/communityapi/v2.0.0"

area_url = "/area/full?AreaId=ZI"


# Establish variable for information desired
zip_code_area = []
prop_tax = []
owner_occupied = []
renter_occupied = []
total_vacant =[]
total_dwellings = []
area_no_data = []
studio_rent = []
one_bed_rent = []
two_bed_rent = []
three_bed_rent = []
four_bed_rent = []


for code in test_zip:
    
    # Make API call
    area_response = requests.post( f'{community_url}{area_url}{code}', headers = headers)
    if area_response.status_code != 200:
        area_no_data.append(area_response.url)
    else:
        area = area_response.json()
        for i in area:
            zip_code_area.append(area['response']['inputparameter']['AreaId'][2:7])
        for i in area['response']['result']['package']['item']:
            prop_tax.append(i['avg_prop_tax'])
            owner_occupied.append(i['dwlowned'])
            renter_occupied.append(i['dwlrent'])
            total_vacant.append(i['dwlvacnt'])
            total_dwellings.append(i['dwltotal'])
            studio_rent.append(i['studio_county'])
            one_bed_rent.append(i['one_bed_county'])
            two_bed_rent.append(i['two_bed_county'])
            three_bed_rent.append(i['three_bed_county'])
            four_bed_rent.append(i['four_bed_county'])

    # Set delay based on API call limitations
    time.sleep(7)

area_data = pd.DataFrame({'zipcode': zip_code_area, 'property_tax': prop_tax,
                        'owner_occupied': owner_occupied, 'renter_occupied': renter_occupied,
                        'total_vacant': total_vacant, 'total_dwellings': total_dwellings,
                        'studio_rent': studio_rent, 'one_bed_rent': one_bed_rent,
                        'two_bed_rent': two_bed_rent, 'three_bed_rent': three_bed_rent,
                        'four_bed_rent': four_bed_rent})
area_data

,zipcode,property_tax,owner_occupied,renter_occupied,total_vacant,total_dwellings,studio_rent,one_bed_rent,two_bed_rent,three_bed_rent,four_bed_rent
0,32608,3629,9028,13308,2920,25256,805,897,1083,1415,1469
1,32601,2350,2101,6151,1125,9377,805,897,1083,1415,1469
2,32603,4152,393,1769,209,2371,805,897,1083,1415,1469


In [160]:
for i in area:
#     print(['inputparameter']['AreaId'])
    print(area['response']['inputparameter']['AreaId'][2:7])

32608


In [111]:
print(area['response']['result']['package']['item'][0]['avg_prop_tax'])

3629


In [112]:
print(area['response']['inputparameter']['AreaId'][2:7])

32608
